In [20]:

import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import random
from sklearn.model_selection import KFold
from load_data import *
from copy import deepcopy
import sys

import torch_geometric

In [21]:

dataset_name = "Coauthor"
data_dir = "../Data/" + dataset_name

total_graph = load_geometric_dataset(dataset_name)
#total_graph = load_blogcatalog(data_dir)
#total_graph = load_toy(data_dir)
#total_graph = load_pubmed(data_dir)
#total_graph = load_flickr(data_dir)
#total_graph = load_reddit(data_dir)

def get_node_features(total_graph, mode="degree"):
    if mode=="degree":
        node_features = np.zeros((total_graph['N_nodes'],2))
        for i in range(total_graph['N_nodes']):
            neighbors = total_graph['edges'][i]
            degree = len(neighbors)
            node_features[i,0] = degree
            second_neighbors_count = 0
            for n in neighbors:
                second_neighbors_count += len(total_graph['edges'][n])
            node_features[i,1] = second_neighbors_count
    elif mode=="degree_dist":
        node_features = total_graph['adj_matrix']/(np.linalg.norm(total_graph['adj_matrix'], axis=1, keepdims=True)+1e-9)
    elif mode=="node_nr":
        node_features = np.zeros((total_graph['N_nodes'],total_graph['N_nodes']))
        for i in range(total_graph['N_nodes']):
            node_features[i,i] = 1
    return node_features

print(total_graph['N_nodes'], total_graph['N_edges'])

18333 163788


In [22]:


class GraphSage(torch.nn.Module):
    def __init__(self, input_dim, output_dim, K):
        super(GraphSage, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, output_dim))
        for i in range(K-1):
            self.layers.append(nn.Linear(output_dim, output_dim))
        for i in range(K):
            nn.init.zeros_(self.layers[i].bias)
        self.K = K
    
    def infer(self, node_features, neighborhood_dict):
        eps = 1e-9
        h = node_features
        N_nodes = h.shape[0]
        for k in range(self.K):
            layer = self.layers[k]
            h_updated = torch.zeros((N_nodes, layer.out_features))
            for v in range(N_nodes):        
                neighborhood = neighborhood_dict[v]
                
                hv = h[v].view(1, -1)
                hN = h[neighborhood]
                conc = torch.cat([hN, hv], dim=0)
                aggregated = torch.mean(conc, dim=0, keepdim=True)
                output = layer(aggregated)
                hv = F.relu(output)
                h_updated[v] =  hv    
            h = h_updated
            h = h / (torch.norm(h, dim=1, keepdim=True)+eps)
        return h

    
    def aggregate_neighbors(self, batch):
        batch_nodes = batch.src_index
        subgraph_edge_indices = batch.edge_index

        neighborhoods = {i.item():[] for i in batch_nodes}
        for src, target in zip(subgraph_edge_indices[0], subgraph_edge_indices[1]):
            src = src.item()
            target = target.item()
            if not neighborhoods.get(src):
                neighborhoods[src] = []
            neighborhoods[src].append(target)
        B = [[] for k in range(self.K+1)]
        B[-1] = batch_nodes[:].tolist()
        for k in range(self.K, 0, -1):
            B[k-1] = B[k][:]
            for node in B[k]:  
                B[k-1].extend(neighborhoods[node])
        return B, neighborhoods


    def forward(self, batch):
        eps = 1e-9
        B, neighborhood_dict = self.aggregate_neighbors(batch)
        h = batch.x
        N_nodes = h.shape[0]
        for k in range(self.K):
            layer = self.layers[k]
            h_updated = torch.zeros((N_nodes, layer.out_features))
            for i,v in enumerate(B[k+1]):      # this is because B[0] is base case, B[1] are nodes corresponding to layer 1 etc  
                neighborhood = neighborhood_dict[v]
                hv = h[v].view(1, -1)
                hN = h[neighborhood]
                conc = torch.cat([hN, hv], dim=0)
                aggregated = torch.mean(conc, dim=0, keepdim=True)
                output = layer(aggregated)
                hv = F.relu(output)
                h_updated[v] =  hv    
            h = h_updated
            h = h / (torch.norm(h, dim=1, keepdim=True)+eps)
        return h



def compute_loss(Z, Z_pos, Z_neg):
    eps = 1e-9
    dot = torch.sum(Z * Z_pos, dim=1)
    term1 = -torch.log(torch.sigmoid(dot)+eps)
    term2 = 0
    for q in range(Z_neg.shape[0]):
        term2 = -torch.log(torch.sigmoid(-torch.sum(Z * Z_neg[q,:,:], dim=1))+eps)
    return torch.mean(term1+term2)



In [23]:
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader
from torch_geometric.sampler import NegativeSampling


feature_type = "node_nr"
node_features_np = get_node_features(total_graph, mode=feature_type)  # to be used for normal datasets
if dataset_name=="toy":
    node_features_np = total_graph['node_feats']    # this is for toy dataset

node_features_torch = torch.tensor(node_features_np).float()
if dataset_name in ["Actor", "Coauthor", "Reddit", "Citeseer"]:
    node_links_torch = total_graph['edges_list']
else:
    node_links_torch = torch.tensor(total_graph['edges_list']).T

print(total_graph['N_nodes'], total_graph['N_edges'])
    
graph_data = torch_geometric.data.Data(node_features_torch, node_links_torch)
# parameters
num_epochs = 1
K = 1   # number of iterations
Q = 10
batch_size = 256
input_size = node_features_np.shape[1]
output_dim = 128
nb_size = 10
directed_graph = False  # used later for LP
# Create the model
model = GraphSage(input_size, output_dim, K)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

ns = NegativeSampling(mode="triplet", amount=Q)
for epoch in range(num_epochs):

    loader = LinkNeighborLoader(graph_data, batch_size=batch_size, num_neighbors=[nb_size]*K, neg_sampling=ns, shuffle=True, replace=True)

    print("starting training")
    for i,batch in enumerate(loader):
        if batch.src_index.shape[0] != batch_size:
            break 
        positive_samples = batch.dst_pos_index.tolist()
        negative_samples = batch.dst_neg_index.numpy()
        Z_tot = model(batch)
        Z = Z_tot[batch.src_index.tolist()]
        Z_pos = Z_tot[positive_samples]
        Z_neg = torch.zeros((Q, batch_size, output_dim))
        for q in range(Q):
            Z_neg[q] = Z_tot[negative_samples[:,q]]
       
        loss = compute_loss(Z, Z_pos, Z_neg)
        print(i/len(loader), loss)
        loss.backward()
        optimizer.step()
    


18333 163788
starting training
0.0 tensor(1.3852, grad_fn=<MeanBackward0>)
0.0015625 tensor(1.3801, grad_fn=<MeanBackward0>)
0.003125 tensor(1.3776, grad_fn=<MeanBackward0>)
0.0046875 tensor(1.3835, grad_fn=<MeanBackward0>)
0.00625 tensor(1.3830, grad_fn=<MeanBackward0>)
0.0078125 tensor(1.3840, grad_fn=<MeanBackward0>)
0.009375 tensor(1.3856, grad_fn=<MeanBackward0>)
0.0109375 tensor(1.3755, grad_fn=<MeanBackward0>)
0.0125 tensor(1.3819, grad_fn=<MeanBackward0>)
0.0140625 tensor(1.3848, grad_fn=<MeanBackward0>)
0.015625 tensor(1.3842, grad_fn=<MeanBackward0>)
0.0171875 tensor(1.3851, grad_fn=<MeanBackward0>)
0.01875 tensor(1.3838, grad_fn=<MeanBackward0>)
0.0203125 tensor(1.3887, grad_fn=<MeanBackward0>)
0.021875 tensor(1.3894, grad_fn=<MeanBackward0>)
0.0234375 tensor(1.3944, grad_fn=<MeanBackward0>)
0.025 tensor(1.3810, grad_fn=<MeanBackward0>)
0.0265625 tensor(1.3953, grad_fn=<MeanBackward0>)
0.028125 tensor(1.3847, grad_fn=<MeanBackward0>)
0.0296875 tensor(1.3875, grad_fn=<MeanBac

In [ ]:

NC_5folds = {}
kf = KFold(n_splits=5, shuffle=True)
nodes = np.array([i for i in range(total_graph['N_nodes'])])
#labels = np.array([total_graph['grops'][n] for n in nodes])
for i, (train_index, test_index) in enumerate(kf.split(nodes)):  
    NC_5folds[i] = {"train":list(nodes[train_index]), "test":list(nodes[test_index])}


In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
from  sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler


def onehot(y, nclasses):
    Y = np.zeros((y.shape[0], nclasses), dtype=int)
    for i in range(y.shape[0]):
        c = y[i]
        Y[i,c-1] =  1
    return Y


def precision_and_recall(Y_true, Y_pred, nclasses):
    # count true positives and false positives and false negatives
    TP_list = [0]*nclasses
    FP_list = [0]*nclasses
    FN_list = [0]*nclasses
    for j in range(nclasses):
       for i, pred in enumerate(Y_pred):
            if pred[j]==1 and Y_true[i][j]==1:
                TP_list[j] += 1
            elif pred[j]==1 and  Y_true[i][j]==0:
                FP_list[j] += 1
            elif pred[j]==0 and Y_true[i][j]==1:
                FN_list[j] += 1 

    return TP_list, FP_list, FN_list

def compute_f1_macro(Y_true, Y_pred, nclasses):
    TP_list, FP_list, FN_list = precision_and_recall(Y_true, Y_pred, nclasses)
    f1_scores = [0]*nclasses
    for k in range(nclasses):
        if TP_list[k]==0:
            continue
        f1_scores[k] = TP_list[k]/(TP_list[k]+0.5*(FP_list[k]+FN_list[k])) 
    return np.sum(f1_scores)/nclasses


def compute_f1_micro(Y_true, Y_pred, nclasses):
    TP_list, FP_list, FN_list = precision_and_recall(Y_true, Y_pred, nclasses)
    TP = np.sum(TP_list)
    FP = np.sum(FP_list)
    FN = np.sum(FN_list)
    print(TP, FP, FN)
    return TP/(TP + 0.5*(FN+FP))

def ind_mapping(nodes):
    mapping = {}
    c = 0
    for node in nodes:
        mapping[node] = c
        c += 1
    return mapping

def compute_neighborhoods_subgraph(edge_dict, nodes, nb_size):
    unique_nodes = set(nodes)
    neighborhoods = {}  
    c = 0
    original_to_new = ind_mapping(nodes)
    for node in nodes:
        neighbors = edge_dict[node]
        # filter out neighbors that are not part of the subgraph   
        neighbors = [original_to_new[n] for n in neighbors if n in unique_nodes] 
        nb = len(neighbors)
        sample_size = min(nb_size, nb)
        if sample_size==0:
            neighborhoods[original_to_new[node]] = []
            continue
        if sample_size == 1:
            sample_neighborhood = [neighbors[0]]*nb_size
        else:
            neighborhood_ind = torch.randint(0, nb, (sample_size,))
            sample_neighborhood = [neighbors[i] for i in neighborhood_ind.tolist()]

        neighborhoods[original_to_new[node]] = sample_neighborhood
        c += 1
    return neighborhoods


N_classes = 15#total_graph['N_classes']
mb = MultiLabelBinarizer(classes=[i for i in range(N_classes)])
f1_macro_list = []
f1_micro_list = []


# 5-fold cross validation
with torch.no_grad():
    for i in range(5):
        print(i)
        training_nodes = torch.tensor(NC_5folds[i]['train'])
        test_nodes = torch.tensor(NC_5folds[i]['test'])
        training_features = node_features_torch[training_nodes]
        test_features = node_features_torch[test_nodes]

        neighborhoods_train = compute_neighborhoods_subgraph(total_graph['edges'], NC_5folds[i]['train'], 25)
        neighborhoods_test = compute_neighborhoods_subgraph(total_graph['edges'], NC_5folds[i]['test'], 25)

        X_train = model.infer(training_features, neighborhoods_train)
        X_test = model.infer(test_features, neighborhoods_test)
        # For the datasets that only have one one label per node, it gives better results to not use multioutputclassifier
        if not total_graph['Multioutput']:
            yt = []
            for n in NC_5folds[i]['train']:
                if len(total_graph['groups'][n]):
                    yt.append(total_graph['groups'][n][0])
                else:
                    yt.append(0)
            #Y_train_sequence = np.array([total_graph['groups'][node][0]  for node in  NC_5folds[i]['train']],dtype=int)
            Y_train_sequence = np.array(yt,dtype=int)
            yt = []
            for n in NC_5folds[i]['test']:
                if len(total_graph['groups'][n]):
                    yt.append(total_graph['groups'][n][0])
                else:
                    yt.append(0)
            #Y_test_sequence = np.array([total_graph['groups'][node][0] for node in  NC_5folds[i]['test'] if len(total_graph['groups'][node])], dtype=int)
            Y_test_sequence = np.array(yt, dtype=int)
            log_reg = LogisticRegression(multi_class="ovr", max_iter=200)
            Y_train = Y_train_sequence
            Y_test = Y_test_sequence
            log_reg.fit(X_train, Y_train)
            Y_pred = log_reg.predict(X_test)
            Y_pred = onehot(Y_pred, N_classes)
            Y_test = onehot(Y_test, N_classes)
        else:
            print("fitting model")
            Y_train_sequence = [total_graph['groups'][node]  for node in NC_5folds[i]['train']]
            Y_test_sequence = [total_graph['groups'][node] for node in NC_5folds[i]['test']]
            Y_train = mb.fit_transform(Y_train_sequence)
            Y_test = mb.fit_transform(Y_test_sequence)
            log_reg = MultiOutputClassifier(SGDClassifier(max_iter=1000))   #multi_class="ovr",
            log_reg.fit(X_train, Y_train)
            Y_pred = log_reg.predict(X_test)
    
        f1_macro = compute_f1_macro(Y_test, Y_pred, N_classes)
        f1_micro = compute_f1_micro(Y_test, Y_pred, N_classes)

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)
        print(f1_macro, f1_micro)

        
    print(np.mean(f1_micro_list))
    print(np.mean(f1_macro_list))

0
372 1148 1148
0.04974966353337549 0.24473684210526317
1
384 1136 1136
0.05052142120651409 0.25263157894736843
2
383 1137 1137
0.055295957959609426 0.2519736842105263
3
358 1162 1162
0.04846327365007754 0.2355263157894737
4
379 1141 1141
0.055493585644607685 0.2493421052631579
0.2468421052631579
0.05190478039883685


In [ ]:
# Select 50% of the edges for training, leave remaining for testing.
# Want the remaining graph to still be connected, so we only remove edges if there are several neighbors

def split_graphs(total_graph, directed=False):
    print("splitting graphs")
    n_test_samples = int(total_graph['N_edges']*0.5)
    training_graph_unbalanced = deepcopy(total_graph["edges"])
    test_graph_unbalanced = {i:[] for i in range(total_graph['N_nodes'])}
    LP_test_X = [(1,1)]*n_test_samples*2
    LP_test_Y = [0]*n_test_samples*2
    counter = 0

    high_degree_nodes = []
    n_neighbors = {i:0 for i in range(total_graph['N_nodes'])}
    for i in range(total_graph['N_nodes']):
        nb_count = len(total_graph['edges'][i]) 
        n_neighbors[i] = nb_count
        if nb_count>1:
            high_degree_nodes.append(i)


    #print(high_degree_nodes[0:10])
    while counter<n_test_samples:
        node1 = random.choice(high_degree_nodes)
        if n_neighbors[node1]>1:
            for node2 in training_graph_unbalanced[node1]:
                if n_neighbors[node2]>1:
                    # Add to test data
                    LP_test_X[counter] = (node1, node2)
                    LP_test_Y[counter] = 1
                    test_graph_unbalanced[node1].append(node2)

                    # remove edge from training graph
                    training_graph_unbalanced[node1].remove(node2)
                       
                    # decrease neighbor count
                    n_neighbors[node1] -= 1

                    # add/remove reverse edge in case of undirected graphs                 
                    if not directed:
                        test_graph_unbalanced[node2].append(node1)
                        try:
                            training_graph_unbalanced[node2].remove(node1)
                        except:
                            print(training_graph_unbalanced[node2], node1, training_graph_unbalanced[node1])
                            raise
                        n_neighbors[node2] -= 1
     

                    counter += 1          
                    if counter%int(n_test_samples/10)==0:
                        print(counter/n_test_samples)
                    break

    return LP_test_X, LP_test_Y, training_graph_unbalanced, test_graph_unbalanced


def balance_test_graph(LP_test_X, LP_test_Y, test_graph_unbalanced, directed=False, reverse_fraction=0.5):
    print("balancing test graph")
    counter = 0
    n_test_samples = int(total_graph['N_edges']*0.5)
    # in case of directed graphs, a fraction of the negative edges are added by reversing true edges
    if directed and reverse_fraction:
        true_edges = LP_test_X[0:n_test_samples]
        while counter<int(n_test_samples*reverse_fraction):
            true_edge = random.choice(true_edges)
            src = true_edge[0]
            target = true_edge[1]
            if not src in test_graph_unbalanced.get(target):
                LP_test_X[n_test_samples+counter] = (target, src)
                counter += 1
            
            if counter%int(n_test_samples/10)==0:
                print(counter/n_test_samples)

    while counter<n_test_samples:
        node1, node2 = random.sample(total_graph['nodes'], 2)
        if not node1 in test_graph_unbalanced[node2]:
            try:
                LP_test_X[n_test_samples+counter] = (node1, node2)
                LP_test_Y[n_test_samples+counter] = 0
            except:
                LP_test_X.append((node1, node2))
                LP_test_Y.append(0)
                print("appended edge")
            counter += 1
    
        if counter%int(n_test_samples/5)==0:
            print(counter/n_test_samples)
    return LP_test_X, LP_test_Y

# When created the test set, we add remaining edges to the training set
# and add negative edges to balance the training data
def balance_training_graph(training_graph_unbalanced, total_graph, directed=False):
    print("balancing training graph")
    n_test_samples = int(total_graph['N_edges']*0.5)
    LP_train_X = []
    LP_train_Y = []
    added_edges = {i:{} for i in range(total_graph['N_nodes'])}
    for node, neighbors in training_graph_unbalanced.items():
        for nb in neighbors:
            if not added_edges[node].get(nb, False):
                added_edges[node][nb] = True
                if not directed:
                    added_edges[nb][node] = True
                LP_train_X.append((node, nb))
                LP_train_Y.append(1)

    n_negative_edges = 0
    while n_negative_edges < n_test_samples:
        node1, node2 = random.sample(total_graph['nodes'], 2)
        if not node1 in training_graph_unbalanced[node2]:
            LP_train_X.append((node1, node2))
            LP_train_Y.append(0)
            n_negative_edges += 1

        if n_negative_edges%int(n_test_samples/10)==0:
            print(n_negative_edges/n_test_samples)
        
    return LP_train_X, LP_train_Y


reverse_fraction = 0
LP_test_X_unb, LP_test_Y_unb, training_graph_unbalanced, test_graph_unbalanced = split_graphs(total_graph, directed=directed_graph)
LP_test_X, LP_test_Y = balance_test_graph(LP_test_X_unb, LP_test_Y_unb, test_graph_unbalanced, directed=directed_graph, reverse_fraction=reverse_fraction)
LP_train_X, LP_train_Y = balance_training_graph(training_graph_unbalanced, total_graph, directed=directed_graph)

splitting graphs
0.09994003597841296
0.19988007195682592
0.29982010793523883
0.39976014391365183
0.4997001798920648
0.5996402158704777
0.6995802518488906
0.7995202878273037
0.8994603238057166
0.9994003597841296
balancing test graph
0.1999466986474782
0.3998933972949564
0.5998400959424346
0.7997867945899128
0.9997334932373909
balancing training graph
0.09994003597841296
0.19988007195682592
0.29982010793523883
0.39976014391365183
0.4997001798920648
0.5996402158704777
0.6995802518488906
0.7995202878273037
0.8994603238057166
0.9994003597841296


In [ ]:
Y_train = LP_train_Y
Y_test = LP_test_Y

from sklearn.metrics import roc_auc_score

def get_edge_representation(fu,fv):
    return torch.sigmoid(torch.dot(fu,fv))


neighborhoods_train = compute_neighborhoods_subgraph(training_graph_unbalanced, total_graph['nodes'], nb_size)
neighborhoods_test = compute_neighborhoods_subgraph(test_graph_unbalanced, total_graph['nodes'], nb_size)
node_features_np = get_node_features(total_graph, mode=feature_type)
node_features_torch = torch.tensor(node_features_np).float()

with torch.no_grad():
    # build representation of edge datasets using inner product of the representation of the two nodes
    X_train = np.zeros((len(LP_train_X), 1))
    Z_train = model.infer(node_features_torch, neighborhoods_train)
    for i, edge in enumerate(LP_train_X):
        u = edge[0]
        v = edge[1]
        X_train[i] = get_edge_representation(Z_train[u], Z_train[v])
    X_test = np.zeros((len(LP_test_X), 1))
    Z_test = model.infer(node_features_torch, neighborhoods_test)
    for i, edge in enumerate(LP_test_X):
        u = edge[0]
        v = edge[1]
        X_test[i] = get_edge_representation(Z_test[u], Z_test[v])
        
    print("fit model")
    classifier = LogisticRegression()
    classifier.fit(X_train, Y_train)
    Y_probs = classifier.predict_proba(X_test)[:,1]
    roc_auc = roc_auc_score(Y_test, Y_probs)
    print(roc_auc)
  

fit model
0.6605648266269324


In [ ]:

with open("../Results/graphsage/{}_metrics{}.csv".format(dataset_name, reverse_fraction), "w") as file:
    settings_str = "Results for graphsage embedding generated with {} epochs, K={}, Q={}, nb_size={}\n".format(num_epochs, K, Q, nb_size)
    file.write(settings_str)
    header = "Dataset; F1 macro; F1 micro; ROC-AUC \n"
    file.write(header)
    data_row = "{dataset};{f1mac};{f1mic};{roc}".format(dataset=dataset_name, f1mac=np.mean(f1_macro_list), f1mic=np.mean(f1_micro_list), roc=roc_auc)
    file.write(data_row)